In [1]:
import os,sys
import numpy as np

In [2]:
# Idea is to calculate the GPU utilization first...
#Denominator would be Total GPU Time (kernel +memory transfers )
# Numerator would be Time spent by GPU doing operations

In [3]:
cuda_kern_sum = 'detailed_profile_cuda_gpu_kern_sum.csv' # contains kernel time
cuda_sum = 'detailed_profile_cuda_gpu_sum.csv' # contains kernel + mem op times
cuda_api_gpu_sum = 'detailed_profile_cuda_api_gpu_sum.csv' # contains kernel + mem ops + API 
nvtx_gpu_proj_sum = 'detailed_profile_nvtx_gpu_proj_sum.csv' # contains total time (gpu + mem ops) annoated code
nvtx_kern_som = 'detailed_profile_nvtx_kern_sum.csv' # contains kernel time

osrt_sum = 'detailed_profile_osrt_sum.csv'

In [4]:
import pandas as pd

In [5]:
def ReadCSV(csv_file):
    df = pd.read_csv(csv_file)
    for col in df.columns:
        df[col] = df[col].replace({',':''},regex=True)
        #Convert if the value is numeric
        try:
            df[col] = df[col].to_numeric(df[col])
        except Exception: 
            pass
    return df
    

In [6]:
df_cuda_kern_sum = ReadCSV(cuda_kern_sum)

In [7]:
df_cuda_kern_sum.keys()

Index(['Time (%)', 'Total Time (ns)', 'Instances', 'Avg (ns)', 'Med (ns)',
       'Min (ns)', 'Max (ns)', 'StdDev (ns)', 'Name'],
      dtype='object')

In [8]:
df_cuda_kern_sum['Name']

0     void at_cuda_detail::cub::DeviceRadixSortSingl...
1     void at::native::index_elementwise_kernel<(int...
2     sm86_xmma_fprop_implicit_gemm_tf32f32_tf32f32_...
3     void at_cuda_detail::cub::DeviceReduceSingleTi...
4     void at::native::elementwise_kernel<(int)128 (...
                            ...                        
86    void at::native::<unnamed>::CatArrayBatchedCop...
87    void at::native::elementwise_kernel<(int)128 (...
88    void at::native::<unnamed>::CatArrayBatchedCop...
89    void at::native::vectorized_elementwise_kernel...
90    void at::native::elementwise_kernel<(int)128 (...
Name: Name, Length: 91, dtype: object

In [9]:
df_cuda_kern_sum['Time ms'] = df_cuda_kern_sum['Total Time (ns)']/1e6
total_gpu_kern_time = df_cuda_kern_sum['Time ms'].sum()

In [10]:
print(total_gpu_kern_time)

281.61836100000005


In [11]:
df_cuda_sum = ReadCSV(cuda_api_gpu_sum)
df_cuda_sum.keys()

Index(['Time (%)', 'Total Time (ns)', 'Instances', 'Avg (ns)', 'Med (ns)',
       'Min (ns)', 'Max (ns)', 'StdDev (ns)', 'Category', 'Operation'],
      dtype='object')

In [12]:
df_cuda_sum['Time ms'] = df_cuda_sum['Total Time (ns)']/1e6
total_gpu_time = df_cuda_sum['Time ms'].sum()
print(total_gpu_time)

tot_gpu_utilization = total_gpu_kern_time/total_gpu_time
print(tot_gpu_utilization)

1085.027863
0.25954942781040824


In [13]:
#25% of the time spent in the GPU (memops + kernel)

In [14]:
df_cuda_sum['Operation']

0                                       cudaLaunchKernel
1                                  cudaStreamSynchronize
2                                        cudaMemcpyAsync
3                                       cuModuleLoadData
4      void at_cuda_detail::cub::DeviceRadixSortSingl...
                             ...                        
115                               cuModuleGetLoadingMode
116    void at::native::vectorized_elementwise_kernel...
117    void at::native::elementwise_kernel<(int)128 (...
118                                        [CUDA memset]
119                       cudaGetDriverEntryPoint_v11030
Name: Operation, Length: 120, dtype: object

In [15]:
df_cuda_sum['Category']

0         CUDA_API
1         CUDA_API
2         CUDA_API
3         CUDA_API
4      CUDA_KERNEL
          ...     
115       CUDA_API
116    CUDA_KERNEL
117    CUDA_KERNEL
118    MEMORY_OPER
119       CUDA_API
Name: Category, Length: 120, dtype: object

In [16]:
df_osrt = ReadCSV(osrt_sum)
df_osrt.keys()

Index(['Time (%)', 'Total Time (ns)', 'Num Calls', 'Avg (ns)', 'Med (ns)',
       'Min (ns)', 'Max (ns)', 'StdDev (ns)', 'Name'],
      dtype='object')

In [17]:
df_osrt['Time ms'] = df_osrt['Total Time (ns)']/1e6
tot_osrt_time = df_osrt['Time ms'].sum()
print(tot_osrt_time)

5695.995998000001


In [18]:
total_gpu_time/tot_osrt_time

np.float64(0.1904895760778236)

In [19]:
df_cuda_time_api = df_cuda_sum[df_cuda_sum['Category']=='CUDA_API']
df_cuda_time_api_ms = df_cuda_time_api['Time ms']

In [20]:
print(df_cuda_time_api_ms.sum())

759.285826


In [21]:
total_gpu_time - df_cuda_time_api_ms.sum()

np.float64(325.742037)

In [22]:
df_cuda_time_kernel = df_cuda_sum[df_cuda_sum['Category']=='CUDA_KERNEL']
df_cuda_time_kernel_ms = df_cuda_time_kernel['Time ms']
print(df_cuda_time_kernel_ms.sum())



281.61836100000005


In [23]:
df_cuda_time_mem = df_cuda_sum[df_cuda_sum['Category']=='MEMORY_OPER']
df_cuda_time_mem_ms = df_cuda_time_mem['Time ms']
print(df_cuda_time_mem_ms.sum())

44.123675999999996


In [24]:
281/(281+759)

0.2701923076923077

In [25]:
df_osrt['Name']

0                       poll
1     pthread_cond_timedwait
2                      ioctl
3                     writev
4                     fclose
5                      fcntl
6                      fread
7                    fopen64
8                      fopen
9             pthread_create
10                     sleep
11                epoll_wait
12                    mmap64
13     pthread_rwlock_wrlock
14                  openat64
15                      read
16                     fgets
17     pthread_rwlock_rdlock
18                      mmap
19             sem_timedwait
20                    open64
21                    fflush
22                      send
23                    socket
24                   connect
25         pthread_cond_wait
26                    munmap
27                      recv
28                   fstat64
29                 epoll_ctl
30        pthread_mutex_lock
31                    fwrite
32                      open
33                     write
34            

In [26]:
(281+44)/4000.


0.08125

In [27]:
5695.995998000001*1e-3


5.695995998000002

In [28]:
df_nvtx_tot = ReadCSV(nvtx_gpu_proj_sum)
df_nvtx_gpu = ReadCSV(nvtx_kern_som)

df_nvtx_tot.keys()

Index(['Range', 'Style', 'Total Proj Time (ns)', 'Total Range Time (ns)',
       'Range Instances', 'Proj Avg (ns)', 'Proj Med (ns)', 'Proj Min (ns)',
       'Proj Max (ns)', 'Proj StdDev (ns)', 'Total GPU Ops', 'Avg GPU Ops',
       'Avg Range Lvl', 'Avg Num Child'],
      dtype='object')

In [29]:
df_nv_range = df_nvtx_tot['Range']
print(df_nv_range)

0          NoTileMPCoincidence::operator()
1           NoTileMPCoincidence::main_loop
2                DNNROIProcess::operator()
3          CCCL:cub::DeviceSelect::Flagged
4                        Decon::operator()
5              Apply1DSpectrum::operator()
6                TorchToTensor::operator()
7                CCCL:cub::DeviceRadixSort
8                ThresholdROIs::operator()
9             TorchFRERSpectrum::configure
10             CCCL:cub::DeviceReduce::Sum
11    NoTileMPCoincidence::gather_channels
12          CCCL:cub::DeviceSelect::Unique
13       FrameToTorchSetFanout::operator()
14            DNNROIPreProcess::preprocess
15      TorchTensorSetCollator::operator()
16                    ApplyROI::operator()
17               Torch1DSpectrum::spectrum
18          DNNROIPostProcess::postprocess
Name: Range, dtype: object


In [30]:
df_nvtx_gpu.keys()

Index(['NVTX Range', 'Style', 'PID', 'TID', 'NVTX Inst', 'Kern Inst',
       'Total Time (ns)', 'Avg (ns)', 'Med (ns)', 'Min (ns)', 'Max (ns)',
       'StdDev (ns)', 'Kernel Name'],
      dtype='object')

In [31]:
print(df_nvtx_gpu['NVTX Range'])

0       Apply1DSpectrum::operator()
1       Apply1DSpectrum::operator()
2       Apply1DSpectrum::operator()
3       Apply1DSpectrum::operator()
4       Apply1DSpectrum::operator()
                   ...             
155       ThresholdROIs::operator()
156       ThresholdROIs::operator()
157       Torch1DSpectrum::spectrum
158    TorchFRERSpectrum::configure
159    TorchFRERSpectrum::configure
Name: NVTX Range, Length: 160, dtype: object


In [32]:
# Group by NVTX Range and sum the Total Time (ns)
gpu_time_by_range = (
    df_nvtx_gpu.groupby("NVTX Range")["Total Time (ns)"]
    .sum()
    .to_dict()
)

In [33]:
gpu_time_by_range

{'Apply1DSpectrum::operator()': 8487020,
 'ApplyROI::operator()': 216033,
 'CCCL:cub::DeviceRadixSort': 29688046,
 'CCCL:cub::DeviceReduce::Sum': 19705617,
 'CCCL:cub::DeviceSelect::Flagged': 30527435,
 'CCCL:cub::DeviceSelect::Unique': 4766454,
 'DNNROIPostProcess::postprocess': 170561,
 'DNNROIPreProcess::preprocess': 708705,
 'DNNROIProcess::operator()': 74173955,
 'Decon::operator()': 7683691,
 'NoTileMPCoincidence::gather_channels': 375841,
 'NoTileMPCoincidence::main_loop': 185063188,
 'NoTileMPCoincidence::operator()': 186889245,
 'ThresholdROIs::operator()': 3129647,
 'Torch1DSpectrum::spectrum': 7104,
 'TorchFRERSpectrum::configure': 1038770}

In [34]:
nvt_tot_proj_time = df_nvtx_tot.groupby('Range')['Total Proj Time (ns)'].sum().to_dict()

In [35]:
nvt_tot_proj_time

{'Apply1DSpectrum::operator()': 33932822,
 'ApplyROI::operator()': 716741,
 'CCCL:cub::DeviceRadixSort': 29688046,
 'CCCL:cub::DeviceReduce::Sum': 19705617,
 'CCCL:cub::DeviceSelect::Flagged': 64168006,
 'CCCL:cub::DeviceSelect::Unique': 10055067,
 'DNNROIPostProcess::postprocess': 179745,
 'DNNROIPreProcess::preprocess': 1088133,
 'DNNROIProcess::operator()': 262999061,
 'Decon::operator()': 53428218,
 'FrameToTorchSetFanout::operator()': 5999488,
 'NoTileMPCoincidence::gather_channels': 10951130,
 'NoTileMPCoincidence::main_loop': 1277255970,
 'NoTileMPCoincidence::operator()': 1363501964,
 'ThresholdROIs::operator()': 28659288,
 'Torch1DSpectrum::spectrum': 398403,
 'TorchFRERSpectrum::configure': 20015433,
 'TorchTensorSetCollator::operator()': 1050788,
 'TorchToTensor::operator()': 30323808}

In [36]:
efficiency = {}
for key in gpu_time_by_range:
    gpu_time_val = gpu_time_by_range[key]
    nvt_tot_proj = nvt_tot_proj_time[key]
    if "CCCL" in key:
        continue
    efficiency[key] = gpu_time_val/nvt_tot_proj*100

In [37]:
print(efficiency)

{'Apply1DSpectrum::operator()': 25.011241328528467, 'ApplyROI::operator()': 30.14101328094807, 'DNNROIPostProcess::postprocess': 94.89053937522601, 'DNNROIPreProcess::preprocess': 65.13036549760002, 'DNNROIProcess::operator()': 28.20312540963787, 'Decon::operator()': 14.381334971718504, 'NoTileMPCoincidence::gather_channels': 3.43198373135923, 'NoTileMPCoincidence::main_loop': 14.489122959433104, 'NoTileMPCoincidence::operator()': 13.706562215116838, 'ThresholdROIs::operator()': 10.920184060399546, 'Torch1DSpectrum::spectrum': 1.7831191030188025, 'TorchFRERSpectrum::configure': 5.189845255908279}


In [38]:
import plotly.express as px
import pandas as pd

In [39]:
df_plot = pd.DataFrame(list(efficiency.items()), columns=["NVTX Range", "GPU Utilization (%)"])
df_plot = df_plot.sort_values("GPU Utilization (%)", ascending=False)

# Plot horizontal bar chart
fig = px.bar(
    df_plot,
    x="GPU Utilization (%)",
    y="NVTX Range",
    orientation="h",
    title="GPU Utilization per NVTX Range",
    text="GPU Utilization (%)",
)
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(yaxis={'categoryorder':'total ascending'}, xaxis_title="GPU Utilization (%)", yaxis_title="NVTX Range")
fig.show()